# Block DCT (Discrete Cosine Transform)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import frame
import block_DCT
import YCoCg as YUV
import cv2
import distortion
import os
import pylab

## Testing `block_DCT.block_analyze()` and `block_DCT.block_synthesize()`

In [ ]:
#a = np.random.randint(low=0, high=100, size=(4,4,3))
a = np.full(shape=(4,4,3), fill_value=10, dtype=np.int16)

In [ ]:
print(a)

In [ ]:
b = block_DCT.block_analyze(a)

In [ ]:
print(b.astype(np.int16))

In [ ]:
c = block_DCT.block_synthesize(b)

In [ ]:
print(c.astype(np.int16))

## Some routines

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    #return 255*(img-min_component)/max_min_component
    return (img-min_component)/max_min_component

def print_stats(frame):
    for i in range(frame.shape[2]):
        print("component", i, frame[..., i].max(), frame[..., i].min(), frame[..., i].dtype)

def show_RGB_frame(frame, title=''):
    plt.figure(figsize=(16,16))
    plt.title(title, fontsize=20)
    plt.imshow(cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_BGR2RGB))
    print_stats(frame)

def show_frame(frame, title=''):
    plt.figure(figsize=(16,16))
    plt.title(title, fontsize=20)
    plt.imshow(frame)
    print_stats(frame)

## Testing `block_DCT.analyze()` and `block_DCT.synthesize()`

In [ ]:
x = frame.read(f"../sequences/stockholm/", 0)
#show_frame(normalize(x))
show_RGB_frame(x, title="Original")

In [ ]:
#x = YUV.from_RGB(x.astype(np.int16))
#show_frame(x)

In [ ]:
block_y_side = block_x_side = 8

In [ ]:
y = block_DCT.analyze(x, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(255*normalize(y), "Block DCT domain")
show_RGB_frame(255*normalize(y[:64, :64]), "Block DCT domain (detail [0:64, 0:64])")

In [ ]:
z = block_DCT.synthesize(y, block_y_side, block_x_side)

In [ ]:
(x-z).max()

In [ ]:
r = x - z
show_RGB_frame(255*normalize(r), "Block DCT finite precission error")

In [ ]:
show_RGB_frame(z, "Reconstructed image")

## Switching between blocks and subbands

In [ ]:
block_y_side = block_x_side = 8
Q_step = 32
x = frame.read(f"../sequences/stockholm/", 0)
y = block_DCT.analyze(x, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, Q_step)
dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, Q_step)
cor_dq_y = block_DCT.get_subbands(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(dq_y, "Decorrelated (original) dequantized block DCT domain")
show_RGB_frame(255*normalize(cor_dq_y), "Correlated dequantized block DCT domain")
blocks_in_y = x.shape[0]//block_y_side
blocks_in_x = x.shape[1]//block_x_side
show_RGB_frame(255*normalize(cor_dq_y[:blocks_in_y, :blocks_in_x]), f"L{block_y_side}L{block_x_side} subband detail")
show_RGB_frame(255*normalize(cor_dq_y[:blocks_in_y, blocks_in_x:2*blocks_in_x]), f"L{block_y_side}H{block_x_side-1} subband detail")
show_RGB_frame(255*normalize(cor_dq_y[blocks_in_y:2*blocks_in_y, :blocks_in_x]), f"L{block_y_side-1}H{block_x_side} subband detail")
show_RGB_frame(255*normalize(cor_dq_y[blocks_in_y:2*blocks_in_y, blocks_in_x:2*blocks_in_x]), f"L{block_y_side-1}H{block_x_side-1} subband detail")

In [ ]:
decor_dq_y = block_DCT.get_blocks(cor_dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(decor_dq_y)

In [ ]:
(decor_dq_y == dq_y).all()

## Testing `block_DCT.constant_quantize()` and `block_DCT.constant_dequantize()`

In [ ]:
block_y_side = block_x_side = 64
q_step = 128
y = block_DCT.analyze(x, block_y_side, block_x_side)
y = block_DCT.get_subbands(y, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, 128)
dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, 128)
dq_y = block_DCT.get_blocks(dq_y, block_y_side, block_x_side)
z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(np.clip(z, a_max=255, a_min=0), "Dequantized (in the block DCT domain and constant Q_step) image")

In [ ]:
r = x - z
n = normalize(r)
show_RGB_frame(255*n, "Quantization error")

## Dynamic range of the subbands

In [ ]:
block_y_side = block_x_side = 256
y = block_DCT.analyze(x, block_y_side, block_x_side)
y = block_DCT.get_subbands(y, block_y_side, block_x_side)
max_, min_ = block_DCT.compute_max_min(y, block_y_side, block_x_side)

In [ ]:
np.set_printoptions(edgeitems=8, linewidth=120, formatter=dict(float=lambda x: "%.3g" % x))
print(max_.astype(np.int32))

In [ ]:
print(min_.astype(np.int32))

## Variances of the subbands

In [ ]:
block_y_side = block_x_side = 256
y = block_DCT.analyze(x, block_y_side, block_x_side)
y = block_DCT.get_subbands(y, block_y_side, block_x_side)
variances = block_DCT.compute_variances(y, block_y_side, block_x_side)

In [ ]:
np.set_printoptions(edgeitems=5, linewidth=100, formatter=dict(float=lambda x: "%.3g" % x))
print(variances.astype(np.int32))
print(variances.max(), np.unravel_index(variances.argmax(), variances.shape))
print(variances.min(), np.unravel_index(variances.argmin(), variances.shape))

## R/D performance

### Subbands vs blocks
For simplicity, we will use constant quantization.

In [ ]:
xx = frame.read(f"../sequences/stockholm/", 0)
x = YUV.from_RGB(xx.astype(np.int16))

disperse_RD_points = []
subband_RD_points = []
block_y_side = block_x_side = 8
for Q_step in [128,64,32,16,8,4,2,1]:
    y = block_DCT.analyze(x, block_y_side, block_x_side)
    # Notice that with constant_quantize() does not matter if the DCT domain
    # is organized in subbands or blocks.
    q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, Q_step)
    dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, Q_step)
    qd_z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)
    dq_zz = YUV.to_RGB(qd_z)
    frame.write(q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    MSE = distortion.MSE(xx, dq_zz)
    disperse_RD_points.append((bits_per_pixel, MSE))
    subband_q_y = block_DCT.get_subbands(q_y, block_y_side, block_x_side)
    frame.write(subband_q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    subband_RD_points.append((bits_per_pixel, MSE))
    print(Q_step, end=' ', flush=True)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*disperse_RD_points), label="disperse B-DCT")
pylab.plot(*zip(*subband_RD_points), label="subband B-DCT")
pylab.title("YCoCg/B-DCT coding performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

## Quantizing depending on the variance
The higher the variance, the lower the quantization step.

In [ ]:
xx = frame.read(f"../sequences/stockholm/", 0)
x = YUV.from_RGB(xx.astype(np.int16))

variance_RD_points = []
block_y_side = block_x_side = 8
for Q_step in [128,64,32,16,8,4,2,1]:
    y = block_DCT.analyze(x, block_y_side, block_x_side)
    y = block_DCT.get_subbands(y, block_y_side, block_x_side)
    variances = np.sqrt(block_DCT.compute_variances(y, block_y_side, block_x_side))
    normalized_variances = (variances - variances.min()) / (variances.max() - variances.min())
    Q_steps = Q_step / np.log(normalized_variances + 1.5)
    print(Q_steps.astype(np.int32))
    q_y = block_DCT.quantize(y, block_y_side, block_x_side, Q_steps)
    dq_y = block_DCT.dequantize(q_y, block_y_side, block_x_side, Q_steps)
    dq_y = block_DCT.get_blocks(dq_y, block_y_side, block_x_side)
    qd_z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)
    dq_zz = YUV.to_RGB(qd_z)
    frame.write(q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    MSE = distortion.MSE(xx, dq_zz)
    variance_RD_points.append((bits_per_pixel, MSE))
    print(Q_step, end=' ', flush=True)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*subband_RD_points), label="constant quantization")
pylab.plot(*zip(*variance_RD_points), label="variance quantization")
pylab.title("YCoCg/B-DCT coding performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

## R/D optimization
The quantization steps should operate in each subband with the same RD-slope.

In [ ]:
xx = frame.read(f"../sequences/stockholm/", 0)
x = YUV.from_RGB(xx.astype(np.int16))

optimal_RD_points = []
block_y_side = block_x_side = 8
for Q_step in [128,64,32,16,8,4,2,1]:
    y = block_DCT.analyze(x, block_y_side, block_x_side)
    y = block_DCT.get_subbands(y, block_y_side, block_x_side)
    #slopes = block_DCT.get_slopes(y, block_y_side, block_x_side, Q_step)
    Q_steps, slopes = block_DCT.find_optimal_Q_steps(y, block_y_side, block_x_side, Q_step)
    print(Q_steps.astype(np.int32))
    q_y = block_DCT.quantize(y, block_y_side, block_x_side, Q_steps)
    dq_y = block_DCT.dequantize(q_y, block_y_side, block_x_side, Q_steps)
    dq_y = block_DCT.get_blocks(dq_y, block_y_side, block_x_side)
    qd_z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)
    dq_zz = YUV.to_RGB(qd_z)
    frame.write(q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    MSE = distortion.MSE(xx, dq_zz)
    optimal_RD_points.append((bits_per_pixel, MSE))
    print(Q_step, end=' ', flush=True)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*variance_RD_points), label="variance quantization")
pylab.plot(*zip(*optimal_RD_points), label="optimal quantization")
pylab.title("YCoCg/B-DCT coding performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

### A special situation

In [ ]:
block_y_side = block_x_side = 256
q_step = 128
y = block_DCT.analyze(x, block_y_side, block_x_side)
y = block_DCT.get_subbands(y, block_y_side, block_x_side)
variances = block_DCT.compute_variances(y, block_y_side, block_x_side)
print(variances.max(), np.unravel_index(variances.argmax(), variances.shape))

## Testing `block_DCT.find_optimal_Q_steps()`

In [ ]:
block_y_side = block_x_side = 64
q_step = 16
y = block_DCT.analyze(x, block_y_side, block_x_side)
y = block_DCT.get_subbands(y, block_y_side, block_x_side)
print(block_DCT.compute_slopes(y, block_y_side, block_x_side, Q_step).astype(np.int32))

In [ ]:
Q_steps, slopes = block_DCT.find_optimal_Q_steps(y, block_y_side, block_x_side, Q_step)

In [ ]:
print(Q_steps, slopes.astype(np.int32))

In [ ]:
q_y = block_DCT.quantize(y, block_y_side, block_x_side, Q_steps)
dq_y = block_DCT.dequantize(q_y, block_y_side, block_x_side, Q_steps)
dq_y = block_DCT.get_blocks(dq_y, block_y_side, block_x_side)
z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(np.clip(z, a_max=255, a_min=0), "Dequantized (using optimal quantization steps) image")

## Computing the variance of the DCT coefficients

In [ ]:
block_y_side = block_x_side = 64
y = block_DCT.analyze(x, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, 128)
v = block_DCT.compute_variances(q_y, block_y_side, block_x_side)

In [ ]:
print(v)

## Quantizing the YCoCg/B-DCT domain

### Using constant quantization

### Using "optimal" quantization

In [ ]:
xx = frame.read(f"../sequences/stockholm/", 0)
x = YUV.from_RGB(xx.astype(np.int16))

constant_RD_points = []
optimal_RD_points = []
block_y_side = block_x_side = 128
for Q_step in [128,64,32,16,8,4,2,1]:
    print(Q_step, end=' ', flush=True)
    y = block_DCT.analyze(x, block_y_side, block_x_side)
    
    # Constant quantization
    q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, Q_step)
    dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, Q_step)
    qd_z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)
    dq_zz = YUV.to_RGB(qd_z)
    subband_q_y = block_DCT.create_subbands(q_y, block_y_side, block_x_side)
    frame.write(subband_q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    MSE = distortion.MSE(xx, dq_zz)
    constant_RD_points.append((bits_per_pixel, MSE))

    # Optimal quantization
    slopes, Q_steps = block_DCT.get_slopes(y, block_y_side, block_x_side, Q_step)
    target_slope = np.median(slopes)
    Q_steps, new_slopes = block_DCT.find_optimal_Q_steps(y, block_y_side, block_x_side, Q_steps, slopes, target_slope)
    print(f"current={slopes.astype(np.int16)}\ntarget_slope={target_slope}\nnew={new_slopes.astype(np.int16)}\nQ_steps={Q_steps}")
    print(slopes.max(), slopes.min())
    print(new_slopes.max(), new_slopes.min())
    q_y = block_DCT.quantize(y, block_y_side, block_x_side, Q_steps)
    dq_y = block_DCT.dequantize(q_y, block_y_side, block_x_side, Q_steps)
    qd_z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)
    dq_zz = YUV.to_RGB(qd_z)
    subband_q_y = block_DCT.create_subbands(q_y, block_y_side, block_x_side)
    frame.write(subband_q_y, f"/tmp/{Q_step}_", 0)
    bits_per_pixel = os.path.getsize(f"/tmp/{Q_step}_000.png")*8/x.size
    MSE = distortion.MSE(xx, dq_zz)
    optimal_RD_points.append((bits_per_pixel, MSE))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*constant_RD_points), label="constant quantized B-DCT")
pylab.plot(*zip(*optimal_RD_points), label="optimal quantized B-DCT")
pylab.title("YCoCg/B-DCT coding performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

In [ ]:
Q_steps

In [ ]:
slopes

In [ ]:
new_slopes